## MealsCount Algorithm (v2)
  
This notebook contains the implementation of variant v2 of an algorithm to groups schools (within a given school district) for maximizing federal funds received through the [**C**ommunity **E**ligiblity **P**rogram](https://www.fns.usda.gov/school-meals/community-eligibility-provision). The groupings generated by the algorithm are near-optimal, optimality being constrained by the need to minimize computational complexity.  
  
For a more detailed understanding of the implementation, containing intermediate results and additional commentary, please take a look at [`mc_algorithm_v2_test`](https://github.com/rkiyengar/mealscount-backend/blob/master/sandbox/mc_algorithm_v2_test.ipynb).  

In [335]:
import os
import sys
import pandas as pd
import numpy as np

import abc

import backend_utils as bu
import config_parser as cp

In [336]:
# display related
from IPython.display import display, HTML

*`mcAlgorithm`* is an abstract base class which is implemented by one or more variants of the MealsCount algorithm. The sole objective of the algorithm is to generate groupings of schools to maximize criteria for CEP eligibility and funding. <u>Currently, *`run`* is designed to be run synchronously and can return the school groups at the end of its execution. However, the API seperates out the generation of school groups and access to them. This is to make room for future changes to allow run to be invoked in an asynchronously.</u>  

In [337]:
class mcAlgorithm(metaclass=abc.ABCMeta):
    """
    Base class for the MealsCount Algorithm. 
    """
            
    def __init__(self):                        
        pass    
    
    @abc.abstractmethod
    def version(self):
        pass
    
    @abc.abstractmethod
    def run(self,df,cfg):
        return True
    
    @abc.abstractmethod
    def get_school_groups(self,df,cfg):
        pass

In [338]:
class CEPSchoolGroupGenerator:
    """
    Implementation of a School District
    """      
    __strategy = None
    
    def __init__(self,cfg,strategy=None):   
        if not (strategy):
            raise ValueError("ERROR: Invalid strategy")
        self.__strategy = strategy
            
    def get_groups(self,df):
        school_groups = {}
        
        if not (self.__strategy):
            raise ValueError("ERROR: Invalid strategy")
        
        try:
            algo = self.__strategy
            if algo.run(df,cfg):
                school_groups = algo.get_school_groups(df,cfg)
            else:
                s = "ERROR: Failed to generate school groups"
                print(s)
            return school_groups
        except Exception as e:            
            raise e            

In [339]:
def prepare_data(df):
    
    # remove aggregated records
    df = df[df['school_name']!='total']
    
    # compute total eligible and isp
    total_eligible = (df['foster'] + df['homeless'] + df['migrant'] + df['direct_cert'])
    isp = (total_eligible/df['total_enrolled']) * 100
    df = df.assign(total_eligible=total_eligible)
    df = df.assign(isp=isp)
    df.loc[:,'isp'] = np.around(df['isp'].astype(np.double),2)
    
    KEEP_COLS = ['school_code','total_enrolled','total_eligible','isp']

    # remove cols not needed for further analysis
    drop_cols = [s for s in df.columns.tolist() if s not in set(KEEP_COLS)]
    df.drop(drop_cols,axis=1,inplace=True)
    
    # sort by isp
    df.sort_values('isp',ascending=False,inplace=True)
    df.reset_index(inplace=True)
    df.drop('index',axis=1,inplace=True)
    
    # compute cumulative isp
    cum_isp = np.around((df['total_eligible'].cumsum()/df['total_enrolled'].cumsum()).astype(np.double)*100,2)
    df = df.assign(cum_isp=cum_isp)
    
    return df

In [340]:
def summarize_group(group_df):
    
        summary = group_df[['total_enrolled','total_eligible']].aggregate(['sum'])        
        summary = summary.assign(grp_isp=round((summary['total_eligible']/summary['total_enrolled'])*100,2))            
        summary = summary.assign(size=group_df.shape[0])
        grp_isp = summary.loc['sum','grp_isp']
        free_rate = round(grp_isp * 1.6,2) if grp_isp >= (cfg.min_cep_thold_pct()*100) else 0.0
        free_rate = 100. if free_rate > 100. else free_rate
        summary = summary.assign(free_rate=free_rate)
        paid_rate = (100.0 - free_rate)
        summary = summary.assign(paid_rate=paid_rate)
        
        return summary

In [341]:
def select_by_isp_impact(df,dst_group_summary,target_isp):
    
    schools_to_add = pd.DataFrame()
    
    dst_grp_total_enrolled = dst_group_summary['total_enrolled']
    dst_grp_total_eligible = dst_group_summary['total_eligible']

    new_total_enrolled = df['total_enrolled'] + dst_grp_total_enrolled
    new_isp = np.around((((df['total_eligible'] + dst_grp_total_eligible)/new_total_enrolled)*100).astype(np.double),2)        
    
    tmp_df = pd.DataFrame({'new_isp':new_isp})
    
    # select all schools whose ISP impact is small enough to not bring down the new ISP 
    # to under the target ISP
    idx = tmp_df[tmp_df['new_isp'] >= target_isp].index
    if len(idx) > 0:
        schools_to_add = df.loc[idx,:]
        
    return schools_to_add

In [342]:
def groupby_isp_width(group_df):
    
    # recalculate cumulative-isp
    group_df = group_df.assign(
        cum_isp=np.around((group_df['total_eligible'].cumsum()/group_df['total_enrolled'].cumsum()).astype(np.double)*100,2))

    top_isp = group_df.iloc[0]['isp']
    
    # if the top percentage is less than that needed for CEP eligibility 
    # we have nothing more to do
    if top_isp < (cfg.min_cep_thold_pct()*100):
        return None
    
    # determine the next cut-off point
    isp_thold = (top_isp - ISP_WIDTH) if (top_isp-ISP_WIDTH) >= (cfg.min_cep_thold_pct()*100) else (cfg.min_cep_thold_pct()*100)
    # group schools at the cut-off point 
    groups = group_df.groupby(pd.cut(group_df['cum_isp'], [0.,isp_thold,top_isp]))    
    
    return groups    

In [343]:
def group_schools_lo_isp(df,cfg):
          
    school_groups = []
    school_group_summaries = []
    
    top_isp = df.iloc[0]['isp']
    
    while top_isp >= (cfg.min_cep_thold_pct()*100):
    
        groups = groupby_isp_width(df)    
    
        if (groups != None):
            
            ivals = pd.DataFrame(groups.size()).index.tolist()
            
            group_df = groups.get_group(ivals[-1]) 
            summary_df = summarize_group(group_df)
            
            df.drop(group_df.index.tolist(),axis=0,inplace=True)                
            schools_to_add = select_by_isp_impact(df,summary_df,(cfg.max_cep_thold_pct()*100))
    
            if schools_to_add.shape[0] > 0:
                group_df = pd.concat([group_df, schools_to_add],axis=0)            
                df.drop(schools_to_add.index.tolist(),axis=0,inplace=True)        
            
            school_groups.append(group_df)
            
            summary_df = summarize_group(group_df)   
            school_group_summaries.append(summary_df)            
            
            # get the top isp for the remaining schools
            top_isp = df.iloc[0]['isp']            

    # the remaining schools are ineligible for CEP 
    # pass this along as a group of its own
    cum_isp = np.around((df['total_eligible'].cumsum()/df['total_enrolled'].cumsum()).astype(np.double)*100,2)
    df = df.assign(cum_isp=cum_isp)        
    school_groups.append(df)
    
    summary_df = summarize_group(df)   
    school_group_summaries.append(summary_df)
    
    return school_groups,school_group_summaries

In [344]:
def group_schools_hi_isp(df,cfg):
    
    school_groups = []
    school_group_summaries = []
    
    # group the data by cumulative ISP such that all schools with
    # max CEP threshold and higher are part of a single group; the rest
    # of the schools are in a second group
    bins = [0.,cfg.max_cep_thold_pct()*100,100.]
    
    groups = df.groupby(pd.cut(df['cum_isp'], bins))
    ivals = groups.size().index.tolist()
    
    group_df = groups.get_group(ivals[-1]).apply(list).apply(pd.Series)    
    summary_df = summarize_group(group_df)
    
    df.drop(group_df.index.tolist(),axis=0,inplace=True)        
    schools_to_add = select_by_isp_impact(df,summary_df,(cfg.max_cep_thold_pct()*100))
    
    if schools_to_add.shape[0] > 0:
        group_df = pd.concat([group_df, schools_to_add],axis=0)
        df.drop(schools_to_add.index.tolist(),axis=0,inplace=True)        
        
    school_groups.append(group_df)
    
    summary_df = summarize_group(group_df)
    school_group_summaries.append(summary_df)
    
    return school_groups,school_group_summaries

In [345]:
def show_results(groups,summaries):    
    
    n = len(groups)
    
    for i in range(n):
        display(HTML('GRP {}'.format(i)))
        display(HTML(summaries[i].to_html()))
        display(HTML(groups[i].to_html()))
        
    return

In [346]:
def prepare_results(groups,summaries):
    
    json_result = {}
    n = len(groups)
    
    #TODO
    for i in range(n):
        pass
            
    return json_result

In [347]:
def runAlgorithmV2(self,df,cfg):
    
    status = True   
        
    df = prepare_data(df)
    
    g1,s1 = group_schools_hi_isp(df,cfg)          
    g2,s2 = group_schools_lo_isp(df,cfg)
        
    self.__school_groups = prepare_results(g1+g2,s1+s2)
    
    # uncomment for debugging
    show_results(g1+g2,s1+s2)
    
    return status

In [348]:
class mcAlgorithmV2(mcAlgorithm):
    """
    Implementation of the MealsCount Algorithm V2
    """
    # school groups JSON data
    __school_groups = {}
            
    def __init__(self):                
        pass
    
    def version(self):
        return "v2"        
    
    def run(self,df,cfg):
        status = self.__run(df,cfg)    
        return status
    
    def get_school_groups(self,df,cfg):
        return self.__school_groups
    
    __run = runAlgorithmV2

### Usage  
  
Below code fragments demonstrate the usage of the above functionality.  

In [349]:
CWD = os.getcwd()

DATADIR = "data"
DATAFILE = "calpads_sample_data.xlsx"

CONFIG_FILE = "config.json"

# FIXME: Add these to the JSON configuration
ALGORITHM_VERSION = "v2"
ISP_WIDTH = 3.125/1.6

In [350]:
school_district_data = bu.mcXLSchoolDistInput(os.path.join(DATADIR,DATAFILE))
df = school_district_data.to_frame()

In [351]:
cfg = cp.mcModelConfig(CONFIG_FILE)

In [352]:
strategy = mcAlgorithmV2() if ALGORITHM_VERSION == "v2" else None

grouper = CEPSchoolGroupGenerator(cfg,strategy)
groups = grouper.get_groups(df)

,total_enrolled,total_eligible,grp_isp,size,free_rate,paid_rate
sum,61,46,75.41,2,100.0,0.0


,school_code,total_enrolled,total_eligible,isp,cum_isp
0,1000001,37,33,89.19,89.19
1,1000027,24,13,54.17,75.41


,total_enrolled,total_eligible,grp_isp,size,free_rate,paid_rate
sum,2439,1222,50.1,5,80.16,19.84


,school_code,total_enrolled,total_eligible,isp,cum_isp
2,1000022,366,190,51.91,51.91
3,1000017,792,407,51.39,51.55
4,1000029,507,258,50.89,51.35
5,1000020,131,66,50.38,51.28
6,1000025,643,301,46.81,50.10


,total_enrolled,total_eligible,grp_isp,size,free_rate,paid_rate
sum,8977,4000,44.56,7,71.3,28.7


,school_code,total_enrolled,total_eligible,isp,cum_isp
7,1000028,2649,1221,46.09,46.09
8,2000002,420,193,45.95,46.07
9,1000011,967,442,45.71,45.99
10,1000014,789,354,44.87,45.80
11,1000006,856,384,44.86,45.66
12,1000004,854,378,44.26,45.48
13,1000024,2442,1028,42.10,44.56


,total_enrolled,total_eligible,grp_isp,size,free_rate,paid_rate
sum,8969,3637,40.55,7,64.88,35.12


,school_code,total_enrolled,total_eligible,isp,cum_isp
14,1000007,2377,990,41.65,41.65
15,1000026,1812,742,40.95,41.35
16,1000005,1628,664,40.79,41.19
17,1000015,1588,634,39.92,40.92
18,2000001,460,182,39.57,40.84
19,2000003,246,96,39.02,40.78
20,1000016,858,329,38.34,40.55


,total_enrolled,total_eligible,grp_isp,size,free_rate,paid_rate
sum,19873,4032,20.29,12,0.0,100.0


,school_code,total_enrolled,total_eligible,isp,cum_isp
21,1000016,1795,665,37.05,37.05
22,1000012,1016,366,36.02,36.68
23,1000013,2089,664,31.79,34.59
24,1000008,77,17,22.08,34.40
25,1000002,1111,224,20.16,31.80
26,1000003,2332,456,19.55,28.41
27,1000021,2403,410,17.06,25.89
28,1000019,2505,394,15.73,23.98
29,1000010,1708,233,13.64,22.81
30,1000018,94,12,12.77,22.74
